# Multiple Access Channel Nonlocality with Two Senders

In [1]:
import pennylane as qml
from pennylane import numpy as np

from context import QNetOptimizer as QNopt

### Classical Bounds for Two-Sender MAC

Linear inequalities bound the set of classical MACs.
The following inequalities have been computed in separate work.


In [2]:
mac_bound_inequalities = [
    (np.array([[0,0,0,0,1,0,1,0,0],[1,1,0,1,0,0,0,0,0]]), 4),
    (np.array([[0,0,1,0,1,0,0,0,0],[1,1,0,1,0,0,0,0,0]]), 4),
    (np.array([[0,0,0,0,0,1,1,0,0],[1,1,0,1,0,0,0,1,0]]), 5),
    (np.array([[0,0,0,0,1,0,0,0,1],[1,1,0,1,0,1,0,0,0]]), 5),
    (np.array([[0,0,1,0,1,0,1,0,0],[2,1,0,1,0,1,0,1,0]]), 7),
    (np.array([[0,0,1,0,1,0,1,0,0],[2,2,0,1,0,0,0,1,0]]), 7),
    (np.array([[0,0,1,0,1,0,1,0,0],[2,1,0,2,0,1,0,0,0]]), 7),
    (np.array([[0,0,0,0,1,1,1,0,1],[1,1,0,1,0,0,0,1,0]]), 6),
    (np.array([[0,0,0,0,1,1,2,0,0],[2,2,0,1,0,0,0,1,0]]), 8),
    (np.array([[0,0,0,0,2,0,1,0,1],[2,2,0,1,0,1,0,0,0]]), 8),
    (np.array([[0,0,1,0,2,0,0,0,1],[2,1,0,2,0,1,0,0,0]]), 8),
    (np.array([[0,0,1,0,2,0,0,0,1],[2,1,0,2,0,0,0,1,0]]), 8),
    (np.array([[0,0,1,0,1,0,2,0,0],[2,1,0,0,0,1,0,2,0]]), 8),
    (np.array([[0,0,1,0,2,0,1,0,1],[3,2,0,2,0,1,0,0,0]]), 10),
    (np.array([[0,0,1,0,2,0,1,0,1],[3,2,0,2,0,0,0,1,0]]), 10),
    (np.array([[0,0,2,0,1,0,2,0,0],[3,1,0,1,0,2,0,2,0]]), 11),
    (np.array([[0,0,2,0,2,0,2,0,0],[3,1,0,1,0,3,0,3,1]]), 14),
    (np.array([[0,0,2,0,3,0,2,0,1],[5,3,0,3,0,1,0,1,0]]), 16),
    (np.array([[0,0,2,1,2,0,5,0,1],[4,2,0,0,0,1,0,4,0]]), 17),
]

In [3]:
def scan_optimize_mac_inequalities(mac_ansatz, **gradient_kwargs):
    for (i,game_tuple) in enumerate(mac_bound_inequalities):

        print("\nMAC inequality id : ", i, ", classical bound : ", game_tuple[1])

        settings = mac_ansatz.rand_scenario_settings()
        cost = QNopt.linear_probs_cost(mac_ansatz, game_tuple[0])
        opt_dict = QNopt.gradient_descent(cost, settings, **gradient_kwargs)

        print("\nmax_score : ", opt_dict["opt_score"])
        print("violation : ", opt_dict["opt_score"] - game_tuple[1])

### Arbitrary Unitary Rotations

In [4]:
arb_prep_nodes = [
    QNopt.PrepareNode(3, [0], qml.templates.subroutines.ArbitraryUnitary, 3),
    QNopt.PrepareNode(3, [1], qml.templates.subroutines.ArbitraryUnitary, 3)
]
arb_meas_nodes = [
    QNopt.MeasureNode(1, 2, [0,1], qml.templates.subroutines.ArbitraryUnitary, 15)
]
arb_mac_ansatz = QNopt.NetworkAnsatz(arb_prep_nodes, arb_meas_nodes)

In [5]:
%%time
np.random.seed(1)
scan_optimize_mac_inequalities(
    arb_mac_ansatz,
    step_size=0.1,
    num_steps=200,
    sample_width=50
)


MAC inequality id :  0 , classical bound :  4
iteration :  0 , score :  2.0526981799219097
iteration :  50 , score :  4.044708747191153
iteration :  100 , score :  4.2532758895515625
iteration :  150 , score :  4.3923750293958035

max_score :  4.409957517462603
violation :  0.4099575174626029

MAC inequality id :  1 , classical bound :  4
iteration :  0 , score :  2.7284866875317366
iteration :  50 , score :  4.278882424628812
iteration :  100 , score :  4.409784244234412
iteration :  150 , score :  4.41408391994293

max_score :  4.41420687279305
violation :  0.41420687279305035

MAC inequality id :  2 , classical bound :  5
iteration :  0 , score :  2.991025780663458
iteration :  50 , score :  4.96401922406987
iteration :  100 , score :  4.997311762196967
iteration :  150 , score :  5.003921733532486

max_score :  5.018693920274231
violation :  0.01869392027423089

MAC inequality id :  3 , classical bound :  5
iteration :  0 , score :  2.766403700067357
iteration :  50 , score :  5.2

### Qubit Rotations on a CNOT 

A simplified ansatz where a CNOT is bookended by y rotation.

In [6]:
local_RY_prep_nodes = [
    QNopt.PrepareNode(3, [0], QNopt.local_RY, 1),
    QNopt.PrepareNode(3, [1], QNopt.local_RY, 1)
]
def CNOT_RY_meas(settings, wires):
    qml.CNOT(wires=wires)
    qml.RY(settings[0],wires=wires[0])
    qml.RY(settings[1],wires=wires[1])

CNOT_RY_meas_nodes = [
    QNopt.MeasureNode(1, 2, [0,1], CNOT_RY_meas, 2)
]

CNOT_RY_mac_ansatz = QNopt.NetworkAnsatz(local_RY_prep_nodes, CNOT_RY_meas_nodes)

In [7]:
%%time

np.random.seed(1)
scan_optimize_mac_inequalities(
    CNOT_RY_mac_ansatz,
    step_size=0.1,
    num_steps=200,
    sample_width=50
)


MAC inequality id :  0 , classical bound :  4
iteration :  0 , score :  2.5139061909613294
iteration :  50 , score :  3.9906857378544585
iteration :  100 , score :  3.9990427818652377
iteration :  150 , score :  4.0026176142248735

max_score :  4.008236632572506
violation :  0.00823663257250562

MAC inequality id :  1 , classical bound :  4
iteration :  0 , score :  2.276794804752609
iteration :  50 , score :  4.001022351890489
iteration :  100 , score :  4.015921324028198
iteration :  150 , score :  4.06297802399126

max_score :  4.214757074554912
violation :  0.2147570745549121

MAC inequality id :  2 , classical bound :  5
iteration :  0 , score :  2.157660258983953
iteration :  50 , score :  5.214474426248964
iteration :  100 , score :  5.242093331063782
iteration :  150 , score :  5.245451069431858

max_score :  5.250472093144684
violation :  0.2504720931446842

MAC inequality id :  3 , classical bound :  5
iteration :  0 , score :  2.0240672684611782
iteration :  50 , score :  4

### Local Rotations do Not Violate MAC Inequalities

In [8]:
local_RY_meas_nodes = [
    QNopt.MeasureNode(1, 2, [0,1], QNopt.local_RY, 2)
]
local_RY_mac_ansatz = QNopt.NetworkAnsatz(local_RY_prep_nodes, local_RY_meas_nodes)

In [9]:
%%time

np.random.seed(1)
scan_optimize_mac_inequalities(
    local_RY_mac_ansatz,
    step_size=0.1,
    num_steps=100,
    sample_width=50
)


MAC inequality id :  0 , classical bound :  4
iteration :  0 , score :  1.5805558330669358
iteration :  50 , score :  3.9940716937533547

max_score :  3.9987530422495228
violation :  -0.001246957750477229

MAC inequality id :  1 , classical bound :  4
iteration :  0 , score :  2.358813704493322
iteration :  50 , score :  3.997579900259851

max_score :  3.9999917213146996
violation :  -8.278685300400213e-06

MAC inequality id :  2 , classical bound :  5
iteration :  0 , score :  1.684782290135559
iteration :  50 , score :  4.996860338735321

max_score :  4.999994863076942
violation :  -5.136923057591503e-06

MAC inequality id :  3 , classical bound :  5
iteration :  0 , score :  4.047631038942107
iteration :  50 , score :  4.99224334183283

max_score :  4.9999749344631335
violation :  -2.5065536866542004e-05

MAC inequality id :  4 , classical bound :  7
iteration :  0 , score :  3.8592877515055366
iteration :  50 , score :  6.9922079679442195

max_score :  6.999571175883788
violation 

### Entanglement Assisted Classical MAC

In [10]:
entanglement_prep_nodes = [
    QNopt.PrepareNode(1, [0,1], qml.templates.subroutines.ArbitraryUnitary, 15)
]
local_arb_meas_nodes = [
    QNopt.MeasureNode(3, 2, [0], qml.templates.subroutines.ArbitraryUnitary, 3),
    QNopt.MeasureNode(3, 2, [1], qml.templates.subroutines.ArbitraryUnitary, 3),
]

entanglement_assisted_mac_ansatz = QNopt.NetworkAnsatz(entanglement_prep_nodes, local_arb_meas_nodes)


In [11]:
%%time

np.random.seed(42)
scan_optimize_mac_inequalities(
    entanglement_assisted_mac_ansatz,
    step_size=0.1,
    num_steps=100,
    sample_width=50
)


MAC inequality id :  0 , classical bound :  4
iteration :  0 , score :  2.5115654912028327
iteration :  50 , score :  4.398294828467266

max_score :  4.414148563590713
violation :  0.4141485635907127

MAC inequality id :  1 , classical bound :  4
iteration :  0 , score :  2.0557812701283
iteration :  50 , score :  4.396631019983337

max_score :  4.413650371179697
violation :  0.4136503711796973

MAC inequality id :  2 , classical bound :  5
iteration :  0 , score :  2.6559495938683337
iteration :  50 , score :  5.26698116212382

max_score :  5.382828756467658
violation :  0.38282875646765824

MAC inequality id :  3 , classical bound :  5
iteration :  0 , score :  2.4696238472190544
iteration :  50 , score :  5.381805630792671

max_score :  5.4124084826338885
violation :  0.4124084826338885

MAC inequality id :  4 , classical bound :  7
iteration :  0 , score :  4.798338676197655
iteration :  50 , score :  7.763876634885358

max_score :  7.777096343359844
violation :  0.777096343359843